<a href="https://colab.research.google.com/github/monk1108/deep_learning/blob/master/5_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()  #先运行nn.Module的初始化函数
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        #nn.Conv2d(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True))
        nn.BatchNorm2d(16),  #参数是层数16
        #归一化处理，这使得数据在进行Relu之前不会因为数据过大而导致网络性能的不稳定
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.fc = nn.Linear(7*7*32, num_classes)  #nn.Linear(in_features, out_features)  
    #一个输入为[batch_size, in_features]的张量变换成了[batch_size, out_features]的输出张量

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    #print("haha")
    #print(out.size())   torch.Size([100, 32, 7, 7])
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)   #fully connected layers
    #print(out.size())   torch.Size([100, 10])
    return out


In [9]:
model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
#model.eval() ：不启用 BatchNormalization 和 Dropout
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.1042
Epoch [1/5], Step [200/600], Loss: 0.2049
Epoch [1/5], Step [300/600], Loss: 0.1304
Epoch [1/5], Step [400/600], Loss: 0.0705
Epoch [1/5], Step [500/600], Loss: 0.1136
Epoch [1/5], Step [600/600], Loss: 0.1666
Epoch [2/5], Step [100/600], Loss: 0.0386
Epoch [2/5], Step [200/600], Loss: 0.0432
Epoch [2/5], Step [300/600], Loss: 0.0657
Epoch [2/5], Step [400/600], Loss: 0.0791
Epoch [2/5], Step [500/600], Loss: 0.0771
Epoch [2/5], Step [600/600], Loss: 0.0606
Epoch [3/5], Step [100/600], Loss: 0.0313
Epoch [3/5], Step [200/600], Loss: 0.0145
Epoch [3/5], Step [300/600], Loss: 0.0187
Epoch [3/5], Step [400/600], Loss: 0.0422
Epoch [3/5], Step [500/600], Loss: 0.0132
Epoch [3/5], Step [600/600], Loss: 0.0414
Epoch [4/5], Step [100/600], Loss: 0.0481
Epoch [4/5], Step [200/600], Loss: 0.0856
Epoch [4/5], Step [300/600], Loss: 0.0530
Epoch [4/5], Step [400/600], Loss: 0.0335
Epoch [4/5], Step [500/600], Loss: 0.0059
Epoch [4/5], Step [600/600], Loss:

In [ ]:
|